In [ ]:
## Explore and sample stock headlines/blurbs and review the model's ability to evaluate the sentiment.
## The purpose of this notebook is to obtain the results of webscraping news headlines and blurbs related to a stock symbol.
## and to evaluate how the model regards the sentiment each text.

In [3]:
import requests
import urllib.request
import time
import spacy
from bs4 import BeautifulSoup
import pandas as pd
from joblib import dump, load

In [4]:
clf = load('sentiment.joblib')

In [35]:
stock_name = "GME"

In [36]:
numResults = 100
url = f'https://www.google.com/search?q=stock+symbol+{stock_name}&tbm=nws&hl=en&num={str(numResults)}'
# https://www.google.com/search?q=stock+symbol+AMZN&tbm=nws&hl=en&num=50

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Get the links, headline and description text
results = soup.find_all("div", attrs={"class": "ZINbbc"})
headlines = []
descriptions = []
links = []
for result in results:
    try:
        headline = result.find("div", attrs={"class": "vvjwJb"}).get_text()
        if headline != "":
            headlines.append(headline)
            links.append("google.com"+result.a["href"])
        description = result.find(
            "div", attrs={"class": "s3v9rd"}).get_text()
        if description != "":
            # remove the "1 day ago" etc.
            description = description[description.find("·")+1:]
            descriptions.append(description)
    except:
        continue

In [47]:
text_list = []
for i in range(0,len(headlines)-1):
    text_list.append(headlines[i]+" "+descriptions[i])

In [48]:
text_list

[' AMC stock (ticker: AMC) closed up 6.4%, at $47.94, on Thursday. GameStop (GME) was up 0.4%, at $191.38. The S&P 500 index was down...',
 ' Among the underlying components of the Russell 3000 index, we saw noteworthy options trading volume today in NCR Corp (Symbol: NCR),...',
 ' The post Buy WISH Stock Before It Joins GME and AMC as a Retail Favorite ... And Bank of America Corp (Symbol: BAC) options are showing a...',
 ' Pachter, 65, who started at Wedbush in 1997, figured the craziness around GameStop (ticker: GME) was winding down in February.',
 ' Sebastian wrote that he recognized that GameStop (ticker: GME) is shifting rapidly from in-store videogame sales to e-commerce, but that he...',
 " It's paid to follow what stocks Reddit's Wall Street Bets crowd are talking about this year, as several have gone through epic ... Ticker: GME",
 ' And GameStop Corp (Symbol: GME) saw options trading volume of 157,110 contracts, representing approximately 15.7 million underlying...',
 ' Gam

In [49]:
df = pd.DataFrame({"News": text_list,
                   "sentiment_no": [clf.predict([text])[0] for text in text_list]})
df

,News,sentiment_no
0,"AMC stock (ticker: AMC) closed up 6.4%, at $4...",0
1,Among the underlying components of the Russel...,1
2,The post Buy WISH Stock Before It Joins GME a...,0
3,"Pachter, 65, who started at Wedbush in 1997, ...",1
4,Sebastian wrote that he recognized that GameS...,0
...,...,...
94,Investors in GameStop Corp (Symbol: GME) saw ...,1
95,... to host a livestream discussing five reas...,1
96,"U.S. stocks jumped on Thursday, roaring back ...",1
97,"Small ""value"" stocks are on fire this year, a...",1


In [50]:
df.sentiment_no.value_counts()

1    58
0    41
Name: sentiment_no, dtype: int64

In [52]:
df.sample(13).to_csv("stock_text_sample.csv")